In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info.operators import Operator
from qiskit.compiler import transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_transpiler_service.transpiler_service import TranspilerService
import numpy as np

In [2]:
QiskitRuntimeService.save_account(channel="ibm_quantum", token="48ee0c8e4e7a0e4b01217f879484b394659c082adb4bda925ef14376f1a72f677e5393bc881f5dee268df994d2668deb8a5722fcefe4a6d10cd5fd7142a2bb53",
                                  set_as_default=True, overwrite=True)
service = QiskitRuntimeService()
#service = QiskitRuntimeService(channel = 'ibm_quantum', instance='ibm-q/open/main', token = '48ee0c8e4e7a0e4b01217f879484b394659c082adb4bda925ef14376f1a72f677e5393bc881f5dee268df994d2668deb8a5722fcefe4a6d10cd5fd7142a2bb53')
#backend = service.get_backend("simulator_mps")
#backend = AerSimulator(method='matrix_product_state')
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=100)
#cloud_transpiler_service = TranspilerService(
#    backend_name=backend.name,
#    ai=False,
#    optimization_level=3
#)

qiskit_runtime_service.__init__:INFO:2024-04-26 01:55:35,086: Default instance: ibm-q/open/main


In [3]:
def gn_gate_controlled(n):
    circ = QuantumCircuit(1)
    #Unitary matrix of G operator
    G = Operator([
        [np.sqrt(1./n), -np.sqrt(1 - 1./n)],
        [np.sqrt(1 - 1./n), np.sqrt(1./n)]])
    #convert into controlled gate
    circ.unitary(G, [0])
    gate = circ.to_gate().control(1)
    return gate

In [4]:
def gn_gate_transpose_controlled(n):
    circ = QuantumCircuit(1)
    #Unitary matrix of G operator
    G = Operator([
        [np.sqrt(1./n), np.sqrt(1 - 1./n)],
        [-np.sqrt(1 - 1./n), np.sqrt(1./n)]])
    #convert into controlled gate
    circ.unitary(G, [0])
    gate = circ.to_gate().control(1)
    return gate

In [5]:
def initialize(circ, row1, row2, row3, row4, ps):
    #set phase shift qubit to |-> by applying X and H gates
    circ.x(ps[0])
    circ.h(ps[0])

    # apply X gate into first qubits of each rows
    circ.x(row1[0])
    circ.x(row2[0])
    circ.x(row3[0])
    circ.x(row4[0])

    #uniform superposition (W state)
    circ = W(circ, row1, row2, row3, row4)

    return circ

In [6]:
def W(circ, row1, row2, row3, row4):
    # controlled-G(n) gates for uniform superposition
    G4 = gn_gate_controlled(4)
    G3 = gn_gate_controlled(3)
    G2 = gn_gate_controlled(2)
    # apply controlled G(n) to 0, 1 quibits of each row
    # then apply CNOT to 1, 0
    # repeat with decremented n
    circ.append(G4, [row1[0], row1[1]])
    circ.append(G4, [row2[0], row2[1]])
    circ.append(G4, [row3[0], row3[1]])
    circ.append(G4, [row4[0], row4[1]])
    circ.cx(row1[1], row1[0])
    circ.cx(row2[1], row2[0])
    circ.cx(row3[1], row3[0])
    circ.cx(row4[1], row4[0])
    circ.append(G3, [row1[1], row1[2]])
    circ.append(G3, [row2[1], row2[2]])
    circ.append(G3, [row3[1], row3[2]])
    circ.append(G3, [row4[1], row4[2]])
    circ.cx(row1[2], row1[1])
    circ.cx(row2[2], row2[1])
    circ.cx(row3[2], row3[1])
    circ.cx(row4[2], row4[1])
    circ.append(G2, [row1[2], row1[3]])
    circ.append(G2, [row2[2], row2[3]])
    circ.append(G2, [row3[2], row3[3]])
    circ.append(G2, [row4[2], row4[3]])
    circ.cx(row1[3], row1[2])
    circ.cx(row2[3], row2[2])
    circ.cx(row3[3], row3[2])
    circ.cx(row4[3], row4[2])

    return circ

In [7]:
def W_inv(circ, row1, row2, row3, row4):
    G4 = gn_gate_transpose_controlled(4)
    G3 = gn_gate_transpose_controlled(3)
    G2 = gn_gate_transpose_controlled(2)

    circ.cx(row1[3], row1[2])
    circ.cx(row2[3], row2[2])
    circ.cx(row3[3], row3[2])
    circ.cx(row4[3], row4[2])
    circ.append(G2, [row1[2], row1[3]])
    circ.append(G2, [row2[2], row2[3]])
    circ.append(G2, [row3[2], row3[3]])
    circ.append(G2, [row4[2], row4[3]])
    circ.cx(row1[2], row1[1])
    circ.cx(row2[2], row2[1])
    circ.cx(row3[2], row3[1])
    circ.cx(row4[2], row4[1])
    circ.append(G3, [row1[1], row1[2]])
    circ.append(G3, [row2[1], row2[2]])
    circ.append(G3, [row3[1], row3[2]])
    circ.append(G3, [row4[1], row4[2]])
    circ.cx(row1[1], row1[0])
    circ.cx(row2[1], row2[0])
    circ.cx(row3[1], row3[0])
    circ.cx(row4[1], row4[0])
    circ.append(G4, [row1[0], row1[1]])
    circ.append(G4, [row2[0], row2[1]])
    circ.append(G4, [row3[0], row3[1]])
    circ.append(G4, [row4[0], row4[1]])

    return circ

In [8]:
def col_cond(circ, row1, row2, row3, row4, col_result):
    # qubit 0 is |1> for each rows
    for i in range(3):
        circ.x(row1[i + 1])
        circ.x(row2[i + 1])
        circ.x(row3[i + 1])
        circ.x(row4[i + 1])
    # AND using toffoli gate
    circ.mcx([row1[0], row1[1], row1[2], row1[3]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3]], col_result[3])
    # qubit 1 is |1> for each rows
    # repeat until qubit 7
    circ.x(row1[0])
    circ.x(row1[1])
    circ.x(row2[0])
    circ.x(row2[1])
    circ.x(row3[0])
    circ.x(row3[1])
    circ.x(row4[0])
    circ.x(row4[1])

    circ.mcx([row1[0], row1[1], row1[2], row1[3],], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3],], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3],], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3],], col_result[3])

    circ.x(row1[1])
    circ.x(row1[2])
    circ.x(row2[1])
    circ.x(row2[2])
    circ.x(row3[1])
    circ.x(row3[2])
    circ.x(row4[1])
    circ.x(row4[2])
    
    circ.mcx([row1[0], row1[1], row1[2], row1[3]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3]], col_result[3])

    circ.x(row1[2])
    circ.x(row1[3])
    circ.x(row2[2])
    circ.x(row2[3])
    circ.x(row3[2])
    circ.x(row3[3])
    circ.x(row4[2])
    circ.x(row4[3])

    circ.mcx([row1[0], row1[1], row1[2], row1[3]], col_result[0])
    circ.mcx([row2[0], row2[1], row2[2], row2[3]], col_result[1])
    circ.mcx([row3[0], row3[1], row3[2], row3[3]], col_result[2])
    circ.mcx([row4[0], row4[1], row4[2], row4[3]], col_result[3])

    #reset all qubits into |1>
    for i in range(3):
        circ.x(row1[i])
        circ.x(row2[i])
        circ.x(row3[i])
        circ.x(row4[i])

    return circ

In [9]:
def dg_cond(circ, row1, row2, row3, row4, dg_result_1, dg_result_2, dg_result_3, dg_result_4):
    # eq3
    # n=3
    circ.ccx(row1[1], row2[0], dg_result_1[0])
    circ.x(dg_result_1[0])

    # n=4
    circ.x(row2[1])
    circ.x(row3[0])

    circ.mcx([row1[2], row2[1], row3[0]], dg_result_1[1])

    circ.x(row1[2])
    circ.x(row2[1])

    circ.mcx([row1[2], row2[1], row3[0]], dg_result_1[1])

    circ.x(row2[1])

    circ.mcx([row1[2], row2[1], row3[0]], dg_result_1[1])

    circ.x(row1[2])
    circ.x(row2[1])
    circ.x(row3[0])

    # n=5
    circ.x(row2[2])
    circ.x(row3[1])
    circ.x(row4[0])

    circ.mcx([row1[3], row2[2], row3[1], row4[0]], dg_result_1[2])

    circ.x(row1[3])
    circ.x(row2[2])

    circ.mcx([row1[3], row2[2], row3[1], row4[0]], dg_result_1[2])

    circ.x(row2[2])
    circ.x(row3[1])

    circ.mcx([row1[3], row2[2], row3[1], row4[0]], dg_result_1[2])

    circ.x(row3[1])

    circ.mcx([row1[3], row2[2], row3[1], row4[0]], dg_result_1[2])

    circ.x(row1[3])
    circ.x(row2[2])
    circ.x(row3[1])
    circ.x(row4[0])

    #-------------------------------------------------------------------------------------------------
    # eq4
    # n=2
    circ.x(row3[2])
    circ.x(row4[1])

    circ.mcx([row2[3], row3[2], row4[1]], dg_result_2[0])

    circ.x(row2[3])
    circ.x(row3[2])

    circ.mcx([row2[3], row3[2], row4[1]], dg_result_2[0])

    circ.x(row4[1])

    circ.mcx([row2[3], row3[2], row4[1]], dg_result_2[0])

    circ.x(row2[3])
    circ.x(row3[2])
    circ.x(row4[1])

    # n=3
    circ.ccx(row3[3], row4[2], dg_result_2[1])
    circ.x(dg_result_2[1])

    #-------------------------------------------------------------------------------------------------
    # eq5
    # n=0
    circ.x(row2[1])
    circ.x(row3[2])
    circ.x(row4[3])

    circ.mcx([row1[0], row2[1], row3[2], row4[3]], dg_result_3[0])

    circ.x(row1[0])
    circ.x(row2[1])

    circ.mcx([row1[0], row2[1], row3[2], row4[3]], dg_result_3[0])

    circ.x(row2[1])
    circ.x(row3[2])

    circ.mcx([row1[0], row2[1], row3[2], row4[3]], dg_result_3[0])

    circ.x(row3[2])

    circ.mcx([row1[0], row2[1], row3[2], row4[3]], dg_result_3[0])

    circ.x(row1[0])
    circ.x(row2[1])
    circ.x(row3[2])
    circ.x(row4[3])

    # n=1
    circ.x(row2[2])
    circ.x(row3[3])

    circ.mcx([row1[1], row2[2], row3[3]], dg_result_3[1])

    circ.x(row1[1])
    circ.x(row2[2])

    circ.mcx([row1[1], row2[2], row3[3]], dg_result_3[1])

    circ.x(row2[2])

    circ.x(row1[1])
    circ.x(row2[2])
    circ.x(row3[3])
    
    # n=2
    circ.ccx(row1[2], row2[3], dg_result_3[2])
    circ.x(dg_result_3[2])

    #-------------------------------------------------------------------------------------------------
    # eq6
    # n=1
    circ.x(row3[1])
    circ.x(row4[2])

    circ.mcx([row2[0], row3[1], row4[2]], dg_result_4[0])

    circ.x(row2[0])
    circ.x(row3[1])

    circ.mcx([row2[0], row3[1], row4[2]], dg_result_4[0])

    circ.x(row3[1])

    circ.mcx([row2[0], row3[1], row4[2]], dg_result_4[0])

    circ.x(row2[0])
    circ.x(row3[1])
    circ.x(row4[2])

    # n=2
    circ.ccx(row3[0], row4[1], dg_result_4[1])
    circ.x(dg_result_4[1])

    return circ

In [10]:
def phase_shift(circ, row1, row2, row3, row4,
                col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps):
    circ.mcx([col_result[0], col_result[1], col_result[2], col_result[3],
            dg_result_1[0], dg_result_1[1], dg_result_1[2],
            dg_result_2[0], dg_result_2[1],
            dg_result_3[0], dg_result_3[1], dg_result_3[2],
            dg_result_4[0], dg_result_4[1]],
            ps)

    return circ

In [11]:
def oracle(circ, row1, row2, row3, row4,
          col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps):
    # column condition
    circ = col_cond(circ, row1, row2, row3, row4, col_result)
    # diagonal condition
    circ = dg_cond(circ, row1, row2, row3, row4,
                  dg_result_1, dg_result_2, dg_result_3, dg_result_4)
    # phase shift
    circ = phase_shift(circ, row1, row2, row3, row4,
                       col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps)
    # apply conditions once more to reset result qubits
    circ = col_cond(circ, row1, row2, row3, row4, col_result)
    circ = dg_cond(circ, row1, row2, row3, row4,
                  dg_result_1, dg_result_2, dg_result_3, dg_result_4)

    return circ

In [12]:
def amplification(circ, row1, row2, row3, row4):
    # apply inverse of operator W
    circ = W_inv(circ, row1, row2, row3, row4)

    # apply X to every inputs
    for i in range(4):
        circ.x(row1[i])
        circ.x(row2[i])
        circ.x(row3[i])
        circ.x(row4[i])

    circ.h(row1[0])

    circ.mcx([row1[1], row1[2], row1[3],
            row2[0], row2[1], row2[2], row2[3],
            row3[0], row3[1], row3[2], row3[3],
            row4[0], row4[1], row4[2], row4[3]],
            row1[0])

    circ.h(row1[0])

    for i in range(4):
        circ.x(row1[i])
        circ.x(row2[i])
        circ.x(row3[i])
        circ.x(row4[i])

    circ = W(circ, row1, row2, row3, row4)

    return circ

In [13]:
def num_operation(N, s):
    num = np.pi / 4 * np.sqrt(np.power(N, N) / s)
    return int(num)

In [ ]:
# main
# 4 rows of 4 quantum registers - total 16
row1 = QuantumRegister(4, name="row1")
row2 = QuantumRegister(4, name="row2")
row3 = QuantumRegister(4, name="row3")
row4 = QuantumRegister(4, name="row4")
# 4 quantum registers to store column condition result
col_result = QuantumRegister(4, name="col_result")
# 4 * 4 - 6 = 10 quantum registers to store diagonal condition result
dg_result_1 = QuantumRegister(3, name="dg_result_1")
dg_result_2 = QuantumRegister(2, name="dg_result_2")
dg_result_3 = QuantumRegister(3, name="dg_result_3")
dg_result_4 = QuantumRegister(2, name="dg_result_4")
# 1 quantum register for phase shift
ps = QuantumRegister(1, name="ps")
# 16 Classical Registers
c_row1 = ClassicalRegister(4, name="row1_measure")
c_row2 = ClassicalRegister(4, name="row2_measure")
c_row3 = ClassicalRegister(4, name="row3_measure")
c_row4 = ClassicalRegister(4, name="row4_measure")
# Circuit
circ = QuantumCircuit(row1, row2, row3, row4,
                      col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps,
                      c_row1, c_row2, c_row3, c_row4)

# uniform superposition
circ = initialize(circ, row1, row2, row3, row4, ps)

# number of operation, 8X8 chessboard with 92 solutions
num_op = num_operation(4, 2)
for _ in range(num_op):
    circ = oracle(circ, row1, row2, row3, row4,
          col_result, dg_result_1, dg_result_2, dg_result_3, dg_result_4, ps)
    circ = amplification(circ, row1, row2, row3, row4)

circ.measure([row1[0], row1[1], row1[2], row1[3],
            row2[0], row2[1], row2[2], row2[3],
            row3[0], row3[1], row3[2], row3[3],
            row4[0], row4[1], row4[2], row4[3]],
            [c_row1[0], c_row1[1], c_row1[2], c_row1[3],
            c_row2[0], c_row2[1], c_row2[2], c_row2[3],
            c_row3[0], c_row3[1], c_row3[2], c_row3[3],
            c_row4[0], c_row4[1], c_row4[2], c_row4[3]])

transpiled_circuit = transpile(circ, backend, optimization_level=3)
#transpiled_circuit = cloud_transpiler_service.run(circ)
job = backend.run(transpiled_circuit)
results = job.result()
answer = results.get_counts()
sorted_answer = sorted(answer)
top_4 = dict(list(sorted_answer.items())[:4])

print(top_4)

INFO:qiskit.passmanager.base_tasks:Pass: UnrollCustomDefinitions - 0.00000 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('u', 1)} to target basis {'x', 'z', 'barrier', 'snapshot', 'p', 'ry', 'measure', 'rz', 'reset', 'delay', 'u', 'cx', 'rx'}.
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation path search completed in 0.000s.
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation paths composed in 0.000s.
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation instructions replaced in 0.000s.
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 0.99993 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnrollCustomDefinitions - 0.50592 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('u', 1)} to target basis {'x', 'z', 'barrier', 'snapshot', 'p', 'ry', 'measure', 'rz', 'reset', 'delay', 'u', 'cx', 'rx'}.
INFO:qiskit.transpiler.pass